# Tugas Pemrograman 3 Pengantar Kecerdasan Buatan
Nama : Reza Ahmad Yahya <br> 
NIM : 1301184403 <br>
Kelas : IF-42-12

## Import Library

In [34]:
import pandas as pd
import math
import numpy as np

## Pembuatan Fungsi yang Dibutuhkan

### 1. Perhitungan Ukuran jarak
Menggunakan <b>Euclidean Distance</b>

In [35]:
def hitungJarak(test, train, row_test):
    arr_dist = []
    for row_train in range(len(train)):
        dist = 0
        for col in range(len(test[row_test])):
            dist += (test[row_test][col] - train[row_train][col])**2

        arr_dist.append([math.sqrt(dist), row_train])
    arr_dist.sort()  
    return arr_dist

### 2. Data Preprocessing

<ol> 
    <li>Algoritma preprocessing pertama mengganti nilai 0 pada <code>kolom = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']</code> dengan nilai mean</li>
    <li>Algoritma preprocessing kedua akan menjadikan data pada satu skala. Algoritma ini mengikuti konsep <b>Normalization</b> dan <b>Standardization</b>. <code>StandardScaler = ( X[i] - mean ) / std</code></li>
</ol>

In [36]:
def cleaning(data):
    kolom = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
    for column in kolom:
        data[column] = data[column].replace(0, np.NaN)
        mean = int(data[column].mean(skipna=True))
        data[column] = data[column].replace(np.NaN, mean)
    return data

In [37]:
def StandardScaler(i, index_data, column_data, key):
    scale = []
    for row in index_data:
        temp = []
        for col in column_data:
            mean = x_train[i][col].mean()
            std = x_train[i][col].std()
            if ( key == 'train'):
                temp.append((x_train[i][col][row] - mean) / std)
            elif ( key == 'test'):
                temp.append((x_test[i][col][row] - mean) / std)
        scale.append(temp)
    return scale

### 3. Klasifikasi KNN
mengklasifikasikan data test yang sedang diprediksi hasilnya untuk mendapatkan hasil akurasi saat dibandingkan dengan data aslinya.

In [38]:
def classification(neighbors, i, y_index):
    class_0 = 0
    class_1 = 0
    for row_neigh in range(len(neighbors)):
        if ( y_train[i][y_index[neighbors[row_neigh][1]]] == 1 ):
            class_1 += 1
        else :
            class_0 += 1
            
    if ( class_0 > class_1 ):
        return 0
    else:
        return 1

### 4. Akurasi Skor
<code>Akurasi = Hasil Benar / Banyak data * 100</code>

In [39]:
def accuracy_score(hasil, i):
    return len(hasil[hasil == True]) / len(y_pred[i]) * 100

### 5. Cek Hasil
untuk membandingkan hasil prediksi dengan data asli

In [40]:
def validasi(i):
    return y_test[i] == y_pred[i]

### 6. Algoritma KNN
<ol> 
    <li> Menghitung Jarak </li>
    <li> Mengambil k tetangga terdekat </li>
    <li> Mengklasifikasi </li>
</ol>

In [53]:
def knn(k):    
    for i in dataset:
        # Inisialisasi array
        arr_pred = []
        y_index = []

        # x_train dan x_test yang sudah di scaling (Preprocessing)
        # terdapat nilai k untuk tiap datasets
        x_trainScale = StandardScaler(i, x_train[i].index, x_train[i].columns, 'train')
        x_testScale = StandardScaler(i, x_test[i].index, x_train[i].columns, 'test')

        # Menyimpan index dataset y_train
        for row in y_train[i].index:
            y_index.append(row)

        # Proses Algoritma KNN untuk mendapatkan nilai Prediksi
        for row_test in range(len(x_testScale)):
            arr_dist = hitungJarak(x_testScale, x_trainScale, row_test)
            neighbors = arr_dist[0:k]
            arr_pred.append(classification(neighbors, i, y_index))

        y_pred[i] = arr_pred
#     k_scores.append(average(y_pred))

## Main Program

In [42]:
# Inisialisasi variabel
filename = 'Diabetes.csv'
data = pd.read_csv(filename)
x_train = {}
x_test = {}
y_train = {}
y_test = {}
y_pred = {}
k_scores = []

data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Pembuatan Lima Datasets Baru
Pertama-tama mengganti nilai-nilai yang 0 pada <code>kolom = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']</code> karena akan memengaruhi <b>outcomenya</b>, ini termasuk <b>Preprocessing Cleaning data</b>. Lalu kita split dulu data menjadi x dan y dengan x berisi semua data dari semua kolom kecuali kolom outcome dan y berisi semua data dari kolom outcome. Setelah itu kita membuat 5 subdatasets berdasarkan berikut: 
1.	Baris ke-1 sampai baris ke-614 sebagai training set dan sisanya sebagai testing set;
2.	Baris ke-1 sampai baris ke-461 ditambah baris ke-615 sampai 768 sebagai training set dan yang lain sebagai testing set;
3.	Baris ke-1 sampai baris ke-307 ditambah baris ke-462 sampai 768 sebagai training set dan yang lain sebagai testing set;
4.	Baris ke-1 sampai baris ke-154 ditambah baris ke-308 sampai 768 sebagai training set dan yang lain sebagai testing set; dan
5.	Baris ke-155 sampai sampai 768 sebagai training set dan yang lain sebagai testing set.

Data tersebut akan menjadi bagian dari <b>K-Fold Cross Validation</b>

In [43]:
# Preprocessing
dataClean = cleaning(data)
dataClean.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,155.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,155.0,26.6,0.351,31,0
2,8,183.0,64.0,29.0,155.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [44]:
# Split x dan y berdasarkan kolom Outcome
x = dataClean.iloc[:, :8]
y = dataClean.iloc[:, 8]

dataset = {
    1 : (614, 768),
    2 : (461, 614),
    3 : (307, 461),
    4 : (154, 307),
    5 : (0, 154)
}

for i in dataset:
    x_train[i] = x.drop([j for j in range(dataset[i][0], dataset[i][1])])
    x_test[i] = x[dataset[i][0]:dataset[i][1]]
    y_train[i] = y.drop([j for j in range(dataset[i][0], dataset[i][1])])
    y_test[i] = y[dataset[i][0]:dataset[i][1]]
    
x_train[1].head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.0,155.0,33.6,0.627,50
1,1,85.0,66.0,29.0,155.0,26.6,0.351,31
2,8,183.0,64.0,29.0,155.0,23.3,0.672,32
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33


In [45]:
k = 26
knn(k)

In [46]:
average = 0
for i in y_pred:
    hasil = validasi(i)
    skor = accuracy_score(hasil, i)
    average += skor
    print('\nDatasets ke- ' + str(i))
    print('Prediksi Benar : ' + str(len(hasil[hasil == True])))
    print('Prediksi Salah : ' + str(len(hasil[hasil == False])))
    print('Skor Akurasi : ' + str(skor) + '%')
print('\nRata-rata Akurasi : ' + str(average/5) +'%')


Datasets ke- 1
Prediksi Benar : 126
Prediksi Salah : 28
Skor Akurasi : 81.81818181818183%

Datasets ke- 2
Prediksi Benar : 121
Prediksi Salah : 32
Skor Akurasi : 79.08496732026144%

Datasets ke- 3
Prediksi Benar : 123
Prediksi Salah : 31
Skor Akurasi : 79.87012987012987%

Datasets ke- 4
Prediksi Benar : 106
Prediksi Salah : 47
Skor Akurasi : 69.28104575163398%

Datasets ke- 5
Prediksi Benar : 117
Prediksi Salah : 37
Skor Akurasi : 75.97402597402598%

Rata-rata Akurasi : 77.20567014684661%


## Best K Program
<code>Note : temporary program, hanya untuk mencari nilai K terbaik dan plottingan untuk kebutuhan data laporan</code>
<br>Perulangan dari k = 3 to k = 30 untuk mencari nilai k terbaik

### Pemilihan Nilai K Terbaik
Fungsi untuk menuntukan nilai K terbaik

In [47]:
# def nilaiK(k_scores):
#     maks = max(k_scores)
#     count = 3
#     for i,j in k_scores:
#         if i == maks[0]:
#             best_k = count
#         else:
#             count+=1

#     return best_k

### Rata-rata Akurasi
menghitung rata-rata akurasi 5-Fold Cross Validation

In [48]:
# def average(prediksi):
#     average = 0
#     hasil = {}
#     for i in dataset:
#         hasil[i] = validasi(i)
#         score = accuracy_score(hasil[i], i)
#         average += score
#     return (average/5, hasil)

In [49]:
# def output(final):  
#     best_k = nilaiK(final)
#     print('Nilai K Terbaik : ' + str(best_k))
#     print('Rata-rata Akurasi : ' + str(final[best_k-3][0]))

#     for i in dataset:
#         hasil = final[best_k-3][1]
#         score = accuracy_score(hasil[i], i)

#         print('\nDatasets ke- ' + str(i))
#         print('Prediksi Benar : ' + str(len(hasil[i][hasil[i] == True])))
#         print('Prediksi Salah : ' + str(len(hasil[i][hasil[i] == False])))
#         print('Skor Akurasi : ' + str(score) + '%')

### Proses pencarian nilai K terbaik
dari <code> k=3 to k=30 </code>

In [50]:
# k_range = range(3, 31)
# for k in k_range:
#     knn(k)

In [51]:
# output(k_scores)

In [52]:
# import matplotlib.pyplot as plt

# x = [i for i in range(3,31)]
# y = []
# xticks = [i*3 for i in range(1, 11)]
# for i in k_scores:
#     y.append(i[0])

# fig, ax = plt.subplots()
# ax.plot(x, y)
# ax.set_xlabel('K [3-30]')
# ax.set_ylabel('Rata-rata Akurasi')
# ax.set_title('Hasil Rata-rata Akurasi tiap K dari [3..30]')
# ax.set_xticks(xticks)
# plt.show()